In [1]:
!pip install torchinfo

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import random
from time import time
import os
import glob

# 파이토치 라이브러리와 필요한 모듈들을 불러옵니다.
import torch #파이토치
import torchvision #파이토치에서 CV 관련 전처리 함수, 데이터 셋 등을 제공하는 라이브러리
import torchinfo

from torch.utils.data import TensorDataset # 텐서데이터셋
from torch.utils.data import DataLoader # 데이터로더

In [3]:
# seed를 고정하는 코드 -> 코드를 다시 돌렸을 때 똑같은 값이 나옴
seed = 2023 # 아무 숫자로 해도됨
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [61]:
# 학습할 때 필요한 변수들을 사전에 정의
# torch에서 학습에 사용할 config variable들을 세팅합니다.
num_workers = 2 #CPU 코어 개수
batch_size = 4
learning_rate = 1e-3
epochs = 10
# -1은 넣으면 안됨

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## 모델에 필요한 데이터 만들기

In [6]:
# 특정 경로에 있는 모든 A301 이름 모두 가져오기
file_pattern = '*.parquet'
file_paths = glob.glob('/content/drive/MyDrive/Colab Notebooks/이어드림 파이널 프로젝트/data/new data/' + file_pattern)

# 파일 이름만 추출, 날짜만 추출
file_name = []
for file_path in file_paths:
    file_name.append(file_path.split('/')[-1])  # 경로에서 파일 이름만 추출

In [7]:
class myscaler:
    def __init__(self):
        self.absmax = None
    def scaler(self,df):
        self.absmax = abs(df).max()
        df = df/abs(df).max()
        return df
    def inverse(self,df):
        df = df*self.absmax
        print(self.absmax)
        return df
sc = myscaler()

In [8]:
# 정답 데이터를 훈련과 테스트로 분할
df = pd.read_parquet('/content/drive/MyDrive/Colab Notebooks/이어드림 파이널 프로젝트/data/target.parquet')
trainY = sc.scaler(df[:169]['외국인']).values
testY = df[169:]['외국인'].values

# 각각의 원소를 리스트로 만든다.
trainY = [[item] for item in trainY]
testY = [[item] for item in testY]

trainY = torch.FloatTensor(trainY)
testY = torch.FloatTensor(testY)
trainY

tensor([[ 5.4870e-01],
        [ 2.6339e-01],
        [ 4.2833e-01],
        [ 1.1706e-01],
        [-4.9885e-01],
        [ 4.6178e-03],
        [ 3.7636e-01],
        [ 2.7953e-01],
        [-1.6713e-01],
        [ 4.1384e-02],
        [ 2.1958e-01],
        [ 1.1410e-01],
        [ 1.2933e-01],
        [ 1.6636e-01],
        [ 2.9693e-02],
        [ 2.7226e-02],
        [ 1.1817e-01],
        [ 5.9047e-02],
        [-5.7432e-02],
        [-4.6099e-03],
        [-1.7194e-02],
        [ 5.1389e-02],
        [-4.7647e-04],
        [-6.9320e-02],
        [-2.6879e-02],
        [ 1.8737e-01],
        [-7.3872e-02],
        [ 9.4838e-02],
        [-4.6714e-02],
        [ 1.1404e-01],
        [-2.6649e-01],
        [-4.1579e-02],
        [ 3.1454e-02],
        [-1.3030e-01],
        [ 6.9736e-02],
        [ 3.0209e-02],
        [ 2.3687e-01],
        [-1.0907e-01],
        [-3.0451e-02],
        [ 1.2593e-01],
        [ 2.9857e-01],
        [ 5.4383e-01],
        [-3.3087e-03],
        [-5

In [9]:
# 학습 데이터를 훈련과 테스트로 분할
trainX_date = file_name[:169] #9월까지를 train으로
testX_date = file_name[169:] #10월부터를 test로

trainX = []
testX =[]
for i in trainX_date:
    df = pd.read_parquet('/content/drive/MyDrive/Colab Notebooks/이어드림 파이널 프로젝트/data/new data/' + i)
    trainX.append(df.transpose().values)
for i in testX_date:
    df = pd.read_parquet('/content/drive/MyDrive/Colab Notebooks/이어드림 파이널 프로젝트/data/new data/' + i)
    testX.append(df.transpose().values)
trainX = torch.FloatTensor(trainX)
testX = torch.FloatTensor(testX)

<ipython-input-9-fc56bda5b707>:13: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:261.)
  trainX = torch.FloatTensor(trainX)


In [10]:
trainX.shape

torch.Size([169, 23, 85000])

In [49]:
trainX

tensor([[[2.0274e-04, 1.0137e-04, 1.0000e+00,  ..., 0.0000e+00,
          0.0000e+00, 0.0000e+00],
         [9.7017e-01, 9.7017e-01, 9.9686e-01,  ..., 0.0000e+00,
          0.0000e+00, 0.0000e+00],
         [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
          0.0000e+00, 0.0000e+00],
         ...,
         [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
          0.0000e+00, 0.0000e+00],
         [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
          0.0000e+00, 0.0000e+00],
         [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
          0.0000e+00, 0.0000e+00]],

        [[3.0191e-05, 2.0128e-04, 2.0128e-04,  ..., 0.0000e+00,
          0.0000e+00, 0.0000e+00],
         [9.9218e-01, 9.9218e-01, 9.9218e-01,  ..., 0.0000e+00,
          0.0000e+00, 0.0000e+00],
         [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
          0.0000e+00, 0.0000e+00],
         ...,
         [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
          0.000

In [12]:
traindataset = TensorDataset(trainX, trainY)
testdataset = TensorDataset(testX, testY)

In [13]:
for i in traindataset:
    print(i)

(tensor([[2.0274e-04, 1.0137e-04, 1.0000e+00,  ..., 0.0000e+00, 0.0000e+00,
         0.0000e+00],
        [9.7017e-01, 9.7017e-01, 9.9686e-01,  ..., 0.0000e+00, 0.0000e+00,
         0.0000e+00],
        [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00, 0.0000e+00,
         0.0000e+00],
        ...,
        [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00, 0.0000e+00,
         0.0000e+00],
        [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00, 0.0000e+00,
         0.0000e+00],
        [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00, 0.0000e+00,
         0.0000e+00]]), tensor([0.5487]))
(tensor([[3.0191e-05, 2.0128e-04, 2.0128e-04,  ..., 0.0000e+00, 0.0000e+00,
         0.0000e+00],
        [9.9218e-01, 9.9218e-01, 9.9218e-01,  ..., 0.0000e+00, 0.0000e+00,
         0.0000e+00],
        [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00, 0.0000e+00,
         0.0000e+00],
        ...,
        [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00, 0.0000e+00,
    

In [62]:
trainloader = DataLoader(traindataset,
                        batch_size=batch_size,
                        shuffle=True,
                        drop_last=True) #데이터 개수가 batch로 깔끔하게 안 나뉠 때 마지막 batch를 버린다.
testloader = DataLoader(testdataset,
                        batch_size=batch_size,
                        shuffle=True,
                        drop_last=True) #데이터 개수가 batch로 깔끔하게 안 나뉠 때 마지막 batch를 버린다.

In [63]:
for i in trainloader:
  print(i)

[tensor([[[1.8530e-03, 3.7059e-05, 4.4471e-05,  ..., 0.0000e+00,
          0.0000e+00, 0.0000e+00],
         [1.0000e+00, 1.0000e+00, 1.0000e+00,  ..., 0.0000e+00,
          0.0000e+00, 0.0000e+00],
         [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
          0.0000e+00, 0.0000e+00],
         ...,
         [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
          0.0000e+00, 0.0000e+00],
         [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
          0.0000e+00, 0.0000e+00],
         [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
          0.0000e+00, 0.0000e+00]],

        [[5.4514e-06, 3.7596e-06, 5.6394e-06,  ..., 0.0000e+00,
          0.0000e+00, 0.0000e+00],
         [1.0000e+00, 1.0000e+00, 1.0000e+00,  ..., 0.0000e+00,
          0.0000e+00, 0.0000e+00],
         [-0.0000e+00, -0.0000e+00, -0.0000e+00,  ..., 0.0000e+00,
          0.0000e+00, 0.0000e+00],
         ...,
         [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
          0

## 모델 정의

In [79]:
import torch.nn as nn  # neural network
# from torch import nn #동일

class CNN1D(nn.Module):
    def __init__(self):
        super().__init__()
        #self.pool = nn.MaxPool1d(kernel_size=2, stride=2) # kernel_size 무조건 기입
        #(23, 58000) ----> (23, 42500)
        self.conv1 = nn.Conv1d(in_channels=23, #들어오는 채널 수(filter의 채널)
                               out_channels=15, #나가는 채널 수(filter의 개수)
                               kernel_size=15, #filter 크기 #(3,3)로 입력가능(동일)
                               stride=2,#(1,1)로 입력가능(동일)
                               padding=0)  # padding 사용하지 않음

        #(23, 58000) ----> (15, 42493) # (23, 42500) ----> (15, 21243)
        self.conv2 = nn.Conv1d(15, 10, 6, 2)  # default: stride=1, padding=0
        #(15, 42493) ----> (10, 21,244) # (15, 21243) ----> (10, 10620)

        self.pool = nn.MaxPool1d(kernel_size=2, stride=2) # kernel_size 무조건 기입
                                         #stride는 default가 kernel_size와 동일한 값
        #(10, 21,244) ----> (10, 10622) # (10, 10620) ----> (10, 5310)
        self.fc1 = nn.Linear(106220, 1024)
        # 23 x 10620(=244260) -----> 131072
        self.fc2 = nn.Linear(1024, 128)
        self.fc3 = nn.Linear(128, 32)
        self.fc4 = nn.Linear(32, 1)
        self.relu = nn.ReLU()

    def forward(self, x):
        #x = self.pool(x)
        x = self.relu(self.conv1(x))
        x = self.relu(self.conv2(x))
        x = self.pool(x)
        x = torch.flatten(x, 1) #1인덱스(두번째) ~ -1인덱스(끝)
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.relu(self.fc3(x))
        x = self.fc4(x)
        return x

In [66]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device) #엔비디아 그래픽 카드가 감지되면 cuda:0 -> 그래픽 카드가 여러개일 수 있어서 0

cuda:0


In [80]:
model = CNN1D().to(device)
#model = CNN1D()

In [18]:
torchinfo.summary(model) #파라미터 개수

Layer (type:depth-idx)                   Param #
CNN1D                                    --
├─MaxPool1d: 1-1                         --
├─Conv1d: 1-2                            5,190
├─Conv1d: 1-3                            610
├─Linear: 1-4                            54,375,424
├─Linear: 1-5                            131,200
├─Linear: 1-6                            2,064
├─Linear: 1-7                            17
├─ReLU: 1-8                              --
Total params: 54,514,505
Trainable params: 54,514,505
Non-trainable params: 0

## Optimizer와 Loss function 정의

In [19]:
import torch.optim as optim

criterion = nn.MSELoss()
#optimizer = optim.SGD(params=model.parameters(), lr=learning_rate)
#optimizer = optim.Adam(model.parameters(), lr=learning_rate)
optimizer = optim.AdamW(model.parameters(), lr=learning_rate)

## 학습

In [81]:
start = time() #학습이 얼마나 걸리는 지 확인 (시작)
from tqdm.auto import tqdm # 진행 사항을 게이지 바로 표시

# for문을 이용하여 epoch마다 학습을 수행하는 코드를 작성합니다.
for epoch in tqdm(range(epochs)):
    # dataloader로 데이터를 불러와서, batch 단위로 데이터를 가져다줌.
    correct = 0
    total_loss = 0.0
    for idx, data in enumerate(trainloader): #반복 1회당 1 batch(3개 데이터)씩 리턴한다.
        images, labels = data[0].to(device), data[1].to(device) #각각 3개씩

        # optimizer의 gradient 초기화.
        optimizer.zero_grad()

        # 모델에 feed-forward를 한 뒤, forward 함수를 실행함.
        outputs = model(images)
        #print(outputs)

        # loss를 계산
        loss = criterion(outputs, labels)  #3개에 대한 loss의 평균
        # 1epoch당 마지막 batch의 loss를 출력
        if idx == len(trainloader)-1:
            print(loss.item())
        # 1epoch에 대한 loss의 평균
        total_loss += loss.item() # 1batch(3)에 대한 loss를 1epoch이 될 때까지 담는다.

        # weight update를 합니다. (back-propagation)
        loss.backward()  # loss를 토대로 gradient 계산
        optimizer.step() # gradient 계산을 토대로 weight update

    print(f"Epoch {epoch+1} ==>  Train loss : {total_loss / len(trainloader):.4f}")

end = time() #학습이 얼마나 걸리는 지 확인 (끝)

print("Training Done.")
print(f"Elasped Time : {end-start:.4f} secs.")

  0%|          | 0/10 [00:00<?, ?it/s]

0.00686267577111721
Epoch 1 ==>  Train loss : 0.0320
0.0863901898264885
Epoch 2 ==>  Train loss : 0.0320
0.00927854422479868
Epoch 3 ==>  Train loss : 0.0322
0.02828071266412735
Epoch 4 ==>  Train loss : 0.0320
0.022403495386242867
Epoch 5 ==>  Train loss : 0.0319
0.01422286406159401
Epoch 6 ==>  Train loss : 0.0317
0.014773843809962273
Epoch 7 ==>  Train loss : 0.0321
0.0032193295191973448
Epoch 8 ==>  Train loss : 0.0321
0.03738703578710556
Epoch 9 ==>  Train loss : 0.0321
0.003929811995476484
Epoch 10 ==>  Train loss : 0.0321
Training Done.
Elasped Time : 10.7670 secs.


## 예측

In [21]:
len(testdataset), len(testloader)
# 출력: (torch.Size([3, 23, 85000]), torch.Size([3, 1])

dataiter = iter(testloader)
images, labels = next(dataiter)
images.shape, labels.shape
# 출력: (torch.Size([3, 23, 85000]), torch.Size([3, 1]))


(torch.Size([3, 23, 85000]), torch.Size([3, 1]))

In [82]:
from sklearn.metrics import mean_absolute_percentage_error
F = []
A = []
with torch.no_grad(): # test time에서 inference할 때, local gradient를 계산하지 않음.
    for data in testloader:
        # GPU version
        images, labels = data[0].to(device), data[1].to(device)

        F_tensor = model(images).cpu().detach()
        F.append([f for f in F_tensor.view(-1).numpy()])

        A_tensor = labels.cpu().detach()
        A.append([a for a in A_tensor.view(-1).numpy()])

F = sc.inverse(np.array(sum(F, [])))
A = np.array(sum(A, []))
mean_absolute_percentage_error(A, F)


13612549


2.435056255004388

In [83]:
F

array([1360524.7876728 , 1366856.62668753, 1361713.95351105,
       1362020.75322625, 1361113.94454738, 1359530.65517417,
       1364894.83267395, 1361570.94934628, 1365267.35345211,
       1363963.78428204, 1363118.03128203, 1358050.5113581 ,
       1361781.09447352, 1357212.56780538, 1362068.0155956 ,
       1362773.29996568])

In [84]:
A

array([-1136106.,   829947., -4255470.,  -748171.,  -373257., -3409660.,
       -3460947., -1368054.,  -420449.,   662348.,  -393700.,  4726550.,
        -433766., -1852188.,  -933840.,  -499400.], dtype=float32)